In [4]:
# Core system dependencies
!apt-get update && apt-get install -y \
    poppler-utils \
    tesseract-ocr

# Required Python packages
!pip install \
    pytesseract \
    pdf2image \
    PyPDF2 \
    pandas \
    openai \
    striprtf

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,317 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,662 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,939 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/

In [5]:
"""
PDF and Vocabulary Text Extractor
--------------------------------
Checkpoint 1: Extract and validate text from input files
"""

import os
import logging
from pathlib import Path
from typing import Dict, List, Tuple
import PyPDF2
from pdf2image import convert_from_path
import pytesseract
from striprtf.striprtf import rtf_to_text
import json

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class TextExtractor:
    def __init__(self, pdf_path: str, vocab_path: str):
        """Initialize paths and directories"""
        self.pdf_path = Path(pdf_path)
        self.vocab_path = Path(vocab_path)

        # Set up output directory
        self.output_dir = Path("extracted_data")
        self.output_dir.mkdir(exist_ok=True)

        # Output file paths
        self.text_output = self.output_dir / "extracted_text.txt"
        self.vocab_output = self.output_dir / "processed_vocabulary.json"
        self.validation_log = self.output_dir / "extraction_validation.log"

        # Initialize processing log
        self.processing_log = []

    def extract_all(self) -> Tuple[str, List[str]]:
        """Main extraction pipeline"""
        try:
            # Process PDF
            pdf_text = self.process_pdf()
            self._log("PDF processing complete")

            # Process vocabulary
            vocabulary = self.process_vocabulary()
            self._log("Vocabulary processing complete")

            # Save outputs
            self.save_outputs(pdf_text, vocabulary)
            self._log("Outputs saved")

            # Validate results
            self.validate_extraction(pdf_text, vocabulary)

            return pdf_text, vocabulary

        except Exception as e:
            self._log_error(f"Extraction failed: {e}")
            raise

    def process_pdf(self) -> str:
        """Process PDF file with readability check"""
        try:
            # Try direct extraction first
            text = self._extract_pdf_text()

            # Check if extracted text is readable
            if self._is_text_readable(text):
                self._log("Direct PDF extraction successful")
                return text

            # If not readable, use OCR
            self._log("Direct extraction insufficient, attempting OCR")
            return self._process_pdf_ocr()

        except Exception as e:
            self._log_error(f"PDF processing failed: {e}")
            raise

    def _extract_pdf_text(self) -> str:
        """Extract text directly from PDF"""
        extracted_text = []

        with open(self.pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)

            for i, page in enumerate(reader.pages):
                text = page.extract_text() or ""
                if text.strip():
                    extracted_text.append(f"--- Page {i+1} ---\n{text}\n")
                self._log(f"Processed page {i+1} directly")

        return "\n".join(extracted_text)

    def _is_text_readable(self, text: str) -> bool:
        """Check if extracted text is readable"""
        if not text.strip():
            return False

        # Check for minimum content
        words = text.split()
        if len(words) < 100:  # Arbitrary threshold
            return False

        # Check for common financial terms
        financial_terms = ['assets', 'liabilities', 'revenue', 'profit', 'loss']
        found_terms = sum(1 for term in financial_terms if term.lower() in text.lower())

        return found_terms >= 2

    def _process_pdf_ocr(self) -> str:
        """Process PDF using OCR"""
        extracted_text = []

        # Convert PDF to images
        images = convert_from_path(str(self.pdf_path))

        for i, image in enumerate(images):
            # Apply OCR with configuration
            text = pytesseract.image_to_string(
                image,
                config='--psm 6 --oem 3'  # Assume uniform text block
            )

            if text.strip():
                extracted_text.append(f"--- Page {i+1} ---\n{text}\n")
            self._log(f"OCR processed page {i+1}")

        return "\n".join(extracted_text)

    def process_vocabulary(self) -> List[str]:
        """Process vocabulary file"""
        try:
            with open(self.vocab_path, 'r') as f:
                content = rtf_to_text(f.read())

            # Clean and extract terms
            terms = [
                term.strip(' ",\\')
                for term in content.split(',')
                if term.strip()
            ]

            # Remove duplicates and sort
            terms = sorted(list(set(
                term for term in terms
                if term and not term.startswith('{')
            )))

            self._log(f"Processed {len(terms)} vocabulary terms")
            return terms

        except Exception as e:
            self._log_error(f"Vocabulary processing failed: {e}")
            raise

    def save_outputs(self, text: str, vocabulary: List[str]):
        """Save extracted data to files"""
        # Save extracted text
        with open(self.text_output, 'w', encoding='utf-8') as f:
            f.write(text)

        # Save vocabulary
        with open(self.vocab_output, 'w', encoding='utf-8') as f:
            json.dump(vocabulary, f, indent=2)

        # Save processing log
        with open(self.validation_log, 'w', encoding='utf-8') as f:
            f.write('\n'.join(self.processing_log))

    def validate_extraction(self, text: str, vocabulary: List[str]):
        """Validate extracted content"""
        validations = []

        # Check text extraction
        validations.append(f"Text length: {len(text)} characters")
        validations.append(f"Number of pages found: {text.count('--- Page')}")

        # Check vocabulary
        validations.append(f"Vocabulary terms: {len(vocabulary)}")
        validations.append(f"Sample terms: {', '.join(vocabulary[:5])}...")

        # Add validations to log
        for validation in validations:
            self._log(f"Validation: {validation}")

    def _log(self, message: str):
        """Log information message"""
        self.processing_log.append(f"INFO: {message}")
        logger.info(message)

    def _log_error(self, message: str):
        """Log error message"""
        self.processing_log.append(f"ERROR: {message}")
        logger.error(message)

def main():
    """Main execution function"""
    try:
        # Initialize extractor
        extractor = TextExtractor(
            pdf_path="/content/vodafone_annual_report.pdf",
            vocab_path="/content/vocabulary.rtf"
        )

        # Process files
        text, vocabulary = extractor.extract_all()

        # Print summary
        print("\n✅ EXTRACTION COMPLETE")
        print(f"\nOutputs saved to: {extractor.output_dir}")
        print(f"- Extracted text: {extractor.text_output}")
        print(f"- Processed vocabulary: {extractor.vocab_output}")
        print(f"- Validation log: {extractor.validation_log}")

    except Exception as e:
        print(f"\n❌ EXTRACTION FAILED: {str(e)}")
        raise

if __name__ == "__main__":
    main()


✅ EXTRACTION COMPLETE

Outputs saved to: extracted_data
- Extracted text: extracted_data/extracted_text.txt
- Processed vocabulary: extracted_data/processed_vocabulary.json
- Validation log: extracted_data/extraction_validation.log


In [8]:
"""
Financial Table Extractor
------------------------
Purpose: Extract financial tables from preprocessed text using GPT-4
"""

import json
import logging
from pathlib import Path
from typing import Dict, List
from google.colab import userdata
import pandas as pd
from openai import OpenAI

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class FinancialTableExtractor:
    def __init__(self, extracted_text_path: str):
        # Initialize OpenAI client with CLY secret
        self.client = OpenAI(api_key=userdata.get('CLY'))

        # Paths
        self.input_path = Path(extracted_text_path)
        self.output_dir = Path("financial_tables")
        self.output_dir.mkdir(exist_ok=True)

    def extract_tables(self) -> Dict[str, pd.DataFrame]:
        """Extract financial tables from text"""
        try:
            # Load extracted text
            with open(self.input_path, 'r', encoding='utf-8') as f:
                text = f.read()

            # Extract tables using GPT-4
            tables = self._extract_tables_gpt(text)

            # Process and save tables
            processed_tables = self._process_tables(tables)

            # Save tables
            self._save_tables(processed_tables)

            return processed_tables

        except Exception as e:
            logger.error(f"Table extraction failed: {e}")
            raise

    def _extract_tables_gpt(self, text: str) -> Dict[str, str]:
        """Extract tables using GPT-4"""
        prompt = """
        Extract these financial tables exactly as they appear in the text:
        1. Balance Sheet / Statement of Financial Position
        2. Income Statement / Profit & Loss Statement
        3. Cash Flow Statement (if present)

        Requirements:
        - Keep ALL line items and numbers EXACTLY as shown
        - Preserve negative values in parentheses
        - Include all subtotals and totals
        - Maintain hierarchy and formatting
        - Keep original labels and capitalization

        For each table:
        - Start with the exact table name as shown in document
        - Include all columns with exact headers
        - Keep numerical formatting exactly as shown

        Format as:
        ### [EXACT TABLE NAME]
        | Line Item | [Year Header 1] | [Year Header 2] |
        |-----------|----------------|----------------|
        | [exact item] | [exact value] | [exact value] |

        Use [TABLE_BREAK] between tables.
        """

        try:
            response = self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {
                        "role": "system",
                        "content": "You are a financial statement expert. Extract tables exactly as they appear, preserving all formatting and details."
                    },
                    {
                        "role": "user",
                        "content": f"{prompt}\n\n{text}"
                    }
                ],
                temperature=0
            )

            # Parse response into tables
            tables = {}
            current_table = None
            current_content = []

            for line in response.choices[0].message.content.split('\n'):
                if line.startswith('### '):
                    if current_table:
                        tables[current_table] = '\n'.join(current_content)
                    current_table = line.replace('### ', '').strip()
                    current_content = []
                elif line.strip():
                    current_content.append(line)

            if current_table and current_content:
                tables[current_table] = '\n'.join(current_content)

            return tables

        except Exception as e:
            logger.error(f"GPT extraction failed: {e}")
            raise

    def _process_tables(self, tables: Dict[str, str]) -> Dict[str, pd.DataFrame]:
        """Convert extracted tables to DataFrames"""
        processed = {}

        for name, content in tables.items():
            try:
                # Parse table content
                rows = [
                    [cell.strip() for cell in row.split('|')[1:-1]]
                    for row in content.split('\n')
                    if row.strip() and '|-' not in row
                ]

                if not rows:
                    continue

                # Create DataFrame preserving exact formatting
                df = pd.DataFrame(rows[1:], columns=rows[0])

                # Clean up any extra whitespace while preserving formatting
                df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

                processed[name] = df

            except Exception as e:
                logger.error(f"Failed to process table {name}: {e}")
                continue

        return processed

    def _save_tables(self, tables: Dict[str, pd.DataFrame]):
        """Save tables in multiple formats"""
        # Save as markdown
        with open(self.output_dir / "financial_tables.md", 'w') as f:
            f.write("# Extracted Financial Tables\n\n")
            for name, df in tables.items():
                f.write(f"## {name}\n")
                f.write(df.to_markdown(index=False))
                f.write("\n\n")

        # Save individual CSV files
        for name, df in tables.items():
            safe_name = "".join(c for c in name if c.isalnum() or c in (' ', '_')).strip()
            df.to_csv(self.output_dir / f"{safe_name}.csv", index=False)

def main():
    try:
        extractor = FinancialTableExtractor(
            extracted_text_path="extracted_data/extracted_text.txt"
        )

        # Extract tables
        tables = extractor.extract_tables()

        # Print results
        print("\n✅ TABLE EXTRACTION COMPLETE")
        print("\nExtracted Tables:")
        for name, df in tables.items():
            print(f"\n## {name}")
            print(df.to_markdown(index=False))

        print(f"\nTables saved to: {extractor.output_dir}")

    except Exception as e:
        print(f"\n❌ EXTRACTION FAILED: {str(e)}")
        raise

if __name__ == "__main__":
    main()


✅ TABLE EXTRACTION COMPLETE

Extracted Tables:

## INCOME STATEMENT
| Line Item                                   | 2020      | 2019      |
|:--------------------------------------------|:----------|:----------|
| Revenue                                     | 5,657.6   | 5,512.9   |
| Cost of sales                               | (4,045.5) | (4,222.2) |
| Gross profit                                | 1,612.1   | 1,290.7   |
| Selling and distribution costs              | (578.1)   | (637.8)   |
| Administrative expenses                     | (1,284.2) | (1,255.0) |
| Net credit losses on financial assets       | (101.3)   | (74.7)    |
| Operating loss                              | (351.5)   | (676.8)   |
| Net finance expense                         | (23.5)    | (2.7)     |
| Loss on ordinary activities before taxation | (375.0)   | (679.5)   |
| Income tax on ordinary activities           | 148.8     | 92.3      |
| Loss for the financial year                 | (226.2)   | (587.2)

In [11]:
"""
Balance Sheet Standardization Processor
-------------------------------------
Purpose: Extract and standardize Balance Sheet using allowed vocabulary terms
"""

import json
import logging
from pathlib import Path
from typing import Dict, Optional, List
from google.colab import userdata
import pandas as pd
from openai import OpenAI
from striprtf.striprtf import rtf_to_text

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class BalanceSheetProcessor:
    def __init__(self):
        # Initialize OpenAI client
        self.client = OpenAI(api_key=userdata.get('CLY'))

        # Set up paths
        self.tables_dir = Path("financial_tables")
        self.vocab_path = Path("/content/vocabulary.rtf")

        # Output paths
        self.output_dir = Path("standardized_output")
        self.output_dir.mkdir(exist_ok=True)

        # Load vocabulary
        self.vocabulary = self._load_vocabulary()
        logger.info(f"Loaded {len(self.vocabulary)} vocabulary terms")

    def _load_vocabulary(self) -> List[str]:
        """Load and clean vocabulary from RTF file"""
        try:
            with open(self.vocab_path, 'r') as f:
                content = rtf_to_text(f.read())
                terms = [
                    term.strip(' ",\\')
                    for term in content.split(',')
                    if term.strip()
                ]
                # Filter and sort terms
                return sorted([t for t in terms if t and not t.startswith('{')])
        except Exception as e:
            logger.error(f"Vocabulary loading failed: {e}")
            raise

    def process_balance_sheet(self):
        """Main processing pipeline"""
        try:
            # Load source tables
            tables = self._load_source_tables()
            logger.info(f"Loaded {len(tables)} tables")

            # Identify and extract Balance Sheet
            balance_sheet = self._get_balance_sheet(tables)
            if balance_sheet is None:
                raise ValueError("Balance Sheet not found in input")

            # Standardize using vocabulary
            standardized = self._standardize_terms(balance_sheet)

            # Generate and save outputs
            self._save_outputs(standardized)

            return standardized

        except Exception as e:
            logger.error(f"Processing failed: {e}")
            raise

    def _load_source_tables(self) -> Dict[str, pd.DataFrame]:
        """Load tables from markdown file"""
        try:
            with open(self.tables_dir / "financial_tables.md", 'r') as f:
                content = f.read()

            # Parse tables
            tables = {}
            current_table = None
            current_lines = []

            for line in content.split('\n'):
                if line.startswith('## '):
                    if current_table and current_lines:
                        tables[current_table] = self._parse_table(current_lines)
                    current_table = line.replace('## ', '').strip()
                    current_lines = []
                elif line.strip():
                    current_lines.append(line)

            if current_table and current_lines:
                tables[current_table] = self._parse_table(current_lines)

            return tables

        except Exception as e:
            logger.error(f"Table loading failed: {e}")
            raise

    def _parse_table(self, lines: List[str]) -> pd.DataFrame:
        """Parse markdown table into DataFrame"""
        try:
            # Get table content
            table_lines = [
                line.strip('|').strip()
                for line in lines
                if '|' in line and '-|-' not in line
            ]

            if not table_lines:
                return pd.DataFrame()

            # Parse headers and data
            headers = [col.strip() for col in table_lines[0].split('|')]
            data = []

            for line in table_lines[1:]:
                cells = [cell.strip() for cell in line.split('|')]
                if len(cells) == len(headers):
                    data.append(cells)

            return pd.DataFrame(data, columns=headers)

        except Exception as e:
            logger.error(f"Table parsing failed: {e}")
            raise

    def _get_balance_sheet(self, tables: Dict[str, pd.DataFrame]) -> Optional[pd.DataFrame]:
        """Extract Balance Sheet table"""
        for name, df in tables.items():
            if any(term.lower() in name.lower()
                  for term in ['balance sheet', 'financial position', 'statement of position']):
                logger.info(f"Found Balance Sheet: {name}")
                return df
        return None

    def _standardize_terms(self, df: pd.DataFrame) -> pd.DataFrame:
        """Standardize Balance Sheet terms using vocabulary"""
        try:
            # Extract line items
            items = df.iloc[:, 0].tolist()

            # Create mapping prompt
            prompt = f"""
            Task: Map these Balance Sheet line items to the standard vocabulary terms.

            Original Balance Sheet items:
            {json.dumps(items, indent=2)}

            Allowed vocabulary terms:
            {json.dumps(self.vocabulary, indent=2)}

            Requirements:
            1. Use ONLY terms from the allowed vocabulary list
            2. Preserve the Balance Sheet structure:
               - Assets (Current and Non-current)
               - Liabilities (Current and Non-current)
               - Equity
            3. Map each term to its closest accounting equivalent
            4. Use section headers (e.g., CURRENT ASSETS, NON CURRENT ASSETS) correctly
            5. Maintain totals and subtotals using correct vocabulary terms

            Return a JSON mapping where:
            - Keys are the original terms
            - Values are the matched allowed terms
            Consider accounting standards and common financial terminology.
            """

            # Get mappings from GPT-4
            response = self.client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {
                        "role": "system",
                        "content": "You are a financial statement expert specializing in accounting terminology standardization."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                temperature=0
            )

            # Apply mappings
            mapping = json.loads(response.choices[0].message.content)
            standardized_df = df.copy()
            standardized_df.iloc[:, 0] = standardized_df.iloc[:, 0].map(lambda x: mapping.get(x, x))

            return standardized_df

        except Exception as e:
            logger.error(f"Term standardization failed: {e}")
            raise

    def _save_outputs(self, df: pd.DataFrame):
        """Save standardized Balance Sheet"""
        try:
            # Prepare markdown content
            content = [
                "# Balance Sheet\n",
                df.to_markdown(index=False),

            ]

            # Save markdown
            output_path = self.output_dir / "standardized_balance_sheet.md"
            with open(output_path, 'w') as f:
                f.write('\n'.join(content))

            # Save CSV
            df.to_csv(self.output_dir / "standardized_balance_sheet.csv", index=False)

            # Display results
            print("\n✅ BALANCE SHEET STANDARDIZATION COMPLETE")
            print("\n Balance Sheet:")
            print(df.to_markdown(index=False))
            print(f"\nOutputs saved to: {self.output_dir}")

        except Exception as e:
            logger.error(f"Output generation failed: {e}")
            raise

def main():
    try:
        processor = BalanceSheetProcessor()
        processor.process_balance_sheet()
    except Exception as e:
        print(f"\n❌ PROCESSING FAILED: {str(e)}")
        raise

if __name__ == "__main__":
    main()


✅ BALANCE SHEET STANDARDIZATION COMPLETE

 Balance Sheet:
| Line Item                                                 | 2020        | 2019        |
|:----------------------------------------------------------|:------------|:------------|
| :-------------------------------------------------------- | :---------- | :---------- |
| NON CURRENT ASSETS                                        |             |             |
| Other Intangible Assets                                   | 2,368.4     | 2,794.1     |
| Land & Buildings                                          | 3,965.3     | 3,040.7     |
| Non Current Related Party Assets                          | 25.1        | 25.1        |
| Provisions and Deferred Taxes                             | 1,124.2     | 1,014.3     |
| Other Non-Current Liabilities                             | 489.2       | 81.4        |
| NON CURRENT ASSETS                                        | 7,972.2     | 6,955.6     |
| CURRENT ASSETS                         

In [12]:
from IPython.display import Markdown, display

# Function to display markdown content in Colab
def display_markdown_from_file(file_path: str):
    try:
        with open(file_path, 'r') as f:
            content = f.read()
        display(Markdown(content))
    except Exception as e:
        print(f"Error displaying markdown: {e}")

# Display the output markdown
display_markdown_from_file('/content/standardized_output/standardized_balance_sheet.md')


# Balance Sheet

| Line Item                                                 | 2020        | 2019        |
|:----------------------------------------------------------|:------------|:------------|
| :-------------------------------------------------------- | :---------- | :---------- |
| NON CURRENT ASSETS                                        |             |             |
| Other Intangible Assets                                   | 2,368.4     | 2,794.1     |
| Land & Buildings                                          | 3,965.3     | 3,040.7     |
| Non Current Related Party Assets                          | 25.1        | 25.1        |
| Provisions and Deferred Taxes                             | 1,124.2     | 1,014.3     |
| Other Non-Current Liabilities                             | 489.2       | 81.4        |
| NON CURRENT ASSETS                                        | 7,972.2     | 6,955.6     |
| CURRENT ASSETS                                            |             |             |
| Inventory (Stock)                                         | 116.4       | 160.9       |
| Accts Rec-Trade (Trade Debtors)                           | 3,582.2     | 3,400.8     |
| Cash                                                      | 243         | 32.5        |
| CURRENT ASSETS                                            | 3,722.9     | 3,594.2     |
| Accts Payable - Trade (Trade Creditors)                   | (4,605.2)   | (4,215.1)   |
| CURRENT LIABILITIES                                       | (882.3)     | (620.9)     |
| TOTAL ASSETS                                              | 7,089.9     | 6,334.7     |
| Long Term Debt                                            | (743.3)     | (21.6)      |
| Provisions and Deferred Taxes                             | (217.1)     | (286.0)     |
| Other Non-Current Liabilities                             | (19.2)      | (178.7)     |
| TOTAL ASSETS                                              | 6,110.3     | 5,848.4     |
| EQUITY                                                    |             |             |
| Share Capital / Paid In Capital                           | -           | -           |
| Other Equity                                              | 9,168.2     | 9,168.2     |
| Retained Earnings                                         | (3,057.9)   | (3,319.8)   |
| TOTAL EQUITY & LIABILITIES                                | 6,110.3     | 5,848.4     |
|                                                           |             |             |